# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [12]:
from urllib import (request, parse)
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [17]:
ETtoday_url = 'https://www.ettoday.net/news/focus/'

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [18]:
browser = webdriver.Firefox(executable_path='/usr/bin/geckodriver')
browser.get(ETtoday_url + parse.quote('政治') + '/')  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 2

time.sleep(SCROLL_PAUSE_TIME)
#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("return document.body.scrollHeight")
while True:
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait to load the page.
    time.sleep(SCROLL_PAUSE_TIME)
    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    '''
    Your code here

    提示：可參考以下的Stack Overflow:
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
last_height

25417

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [22]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    html = request.urlopen(link_url)
    response = BeautifulSoup(html.read().decode('utf-8'))
    news_content = response.find("div", class_='story').find_all("p")
    p_list = []
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if p.string is not None:
            p_list.append(p.string)
    return p_list

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [25]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html)
news_block = soup.find("div", class_='part_pictxt_3 lazyload')
all_news = news_block.find_all('div', attrs={'class':'piece clearfix'})
news_list = []
for news_item in all_news:
    news_body = news_item.find('h3')
    news_list.append(dict(title = news_body.a.string))
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    news_list[-1]['content'] = getNewsDetailContent(externalLink)
news_list

[{'title': '成立政治獻金專戶！前議員王長明角逐連江立委「為弱勢發聲」',
  'content': ['▲王長明在提出的政見中強調，疼惜自己就要重視弱勢。（圖／翻攝王長明臉書）',
   '記者洪珮瑄／綜合報導',
   '2020大選倒數16天，連江縣前議員王長明這次也決定角逐立委一位，並且成立政治獻金專戶，開始對外募款，表達參選決心。除了改善馬祖外來的醫療資源外，他也秉持著「重視弱勢」的理念，希望能為一些有經濟困難的人盡一份心力。',
   '據悉，王長明在今年1月宣布退出民進黨，此次以「無黨籍」參選連江立委選舉。他在馬祖資訊網論壇上表示，自從戰地政務時期投身民主政治以來，已歷時30多年，自己在縣議員選舉失利後，原本考慮要淡出政治，但基於一些考量，「決定再做一次馬前卒」。',
   '王長明指出，這些年來，念念不忘的民防自衛隊員榮民就養，目前僅發給慰助金，需要有人去為他們爭取完整的權利。此外，他也提到，民進黨新人李問此次角逐連江縣立委，但歷屆選舉結果顯示，新人空降的得票有限，對現任陳雪生立委無法產生壓力和監督效果。',
   '▲王長明指出，投身民主政治以來已30多年，這次決定挑戰立委選舉，打破馬祖困境。（圖／翻攝王長明臉書）',
   '為了打破目前馬祖的困境，王長明決定參選立委，並向監察院和馬祖郵局申請政治獻金專戶，並開始對外募款。他表示，自己目前仍在山隴獅子市場擺攤，因此經濟有限，呼籲鄉親能夠支持。',
   '走過軍事管制前後30多年的歲月，王長明希望能夠重建馬祖純樸、公正、公平的選風，並努力讓中央對馬祖更加重視，協助解決馬祖土地問題。他強調，自己的參選也讓選民在藍、綠之外，多一個選擇。',
   '▼王長明為了這次的選舉，成立政治獻金專戶。（圖／翻攝馬祖資訊網論壇）']},
 {'title': '政見會談蕉農眼眶泛紅\u3000宋楚瑜：我不是激動是感同身受',
  'content': ['記者林銘翰／台北報導',
   '2020總統大選第三場政見會27日晚間登場，總統蔡英文、國民黨總統候選人韓國瑜、親民黨總統候選人宋楚瑜三度交鋒，宋楚瑜於第二輪發言中，談及香蕉價格慘跌、蕉農割蕉餵豬時忍不住眼眶泛紅，痛批《反滲透法》斷了農民財路。他會後受訪時表示，自己不是激動、而是一種有同樣感受的感覺，因為如果政治人物對人民的痛苦無動於衷，「坦白講他最